In [12]:
# Libraries Imports
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [27]:
## Gather Data
boston_data = load_boston()
data = pd.DataFrame(data = boston_data.data, columns=boston_data.feature_names)
data['PRICE'] = boston_data.target
dir(boston_data)
print(boston_data.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [14]:
features = data.drop(columns=['PRICE',"AGE",'INDUS'])
target = np.log(data['PRICE'])

In [15]:
regressor = LinearRegression()
regressor.fit(features, target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [20]:
variance = mean_squared_error(target, regressor.predict(features))
standard_deviation = np.sqrt(variance)

## Estimation

In [38]:
CRIM_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8
property_stats = features.mean().values.reshape(1, 11)

In [44]:
property_stats.shape

(1, 11)

In [46]:
def get_log_estimate(nr_rooms, ptratio, chas = True, high_confidence = True):
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = ptratio
    if chas:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
    
    predict_log_price = regressor.predict(property_stats)[0]
    if high_confidence:
        upper_bound = predict_log_price + 2 * standard_deviation
        lower_bound = predict_log_price - 2 * standard_deviation
        interval = 95
    else:
        upper_bound = predict_log_price + standard_deviation
        lower_bound = predict_log_price - standard_deviation
        interval = 68
    return(predict_log_price, upper_bound, lower_bound, interval)

In [49]:
get_log_estimate(3, 20, chas=True, high_confidence=False)

(2.7767581914803996, 2.96427032667753, 2.5892460562832693, 68)

In [54]:
def get_real_estimate(num_rooms, student_teacher_ratio, next_to_river = True, large_range = True):
    log_est, log_upp, log_low, inter = get_log_estimate(nr_rooms = num_rooms, ptratio = student_teacher_ratio,
                                                        chas = next_to_river, high_confidence = large_range)
    actual_price = (np.e**log_est)*1000
    actual_upp = (np.e**log_upp)*1000
    actual_low = (np.e**log_low)*1000
    
    #Rounding
    round_est = np.round(actual_price)
    round_upp = np.round(actual_upp)
    round_low = np.round(actual_low)
    
    #Printing
    print(f"The estimated price of house is: USD {round_est}" )
    print(f'At {inter}% confidence the valuation range is')
    print(f'USD {round_low} at the lower end to USD {round_upp} at the high end.')

In [55]:
get_real_estimate(3, 20, next_to_river=True, large_range=False)

The estimated price of house is: USD 16067.0
At 68% confidence the valuation range is
USD 13320.0 at the lower end to USD 19381.0 at the high end.


In [56]:
get_real_estimate(3, 20, next_to_river=True, large_range=True)

The estimated price of house is: USD 16067.0
At 95% confidence the valuation range is
USD 11042.0 at the lower end to USD 23378.0 at the high end.
